# Lexicase Selection Demo

A simple demonstration of lexicase selection for parent selection in evolutionary computation.


In [1]:
import numpy as np
import lexicase


## Create Sample Data

We'll create a fitness matrix where each row represents an individual and each column represents a test case. Higher values indicate better performance.


In [4]:
# Fitness matrix (4 individuals × 5 test cases)
fitness_matrix = np.array([
    [8.5, 6.2, 9.1, 4.3, 7.8],  # Individual 0
    [7.1, 9.4, 5.6, 8.2, 6.9],  # Individual 1
    [9.2, 5.8, 7.4, 9.1, 8.5],  # Individual 2
    [6.3, 8.7, 6.8, 7.5, 9.2]   # Individual 3
])

print("Fitness Matrix:")
print(fitness_matrix)
print(f"\nShape: {fitness_matrix.shape[0]} individuals × {fitness_matrix.shape[1]} test cases")


Fitness Matrix:
[[8.5 6.2 9.1 4.3 7.8]
 [7.1 9.4 5.6 8.2 6.9]
 [9.2 5.8 7.4 9.1 8.5]
 [6.3 8.7 6.8 7.5 9.2]]

Shape: 4 individuals × 5 test cases


## Standard Lexicase Selection


In [5]:
# Select 6 individuals using standard lexicase
selected = lexicase.lexicase_selection(
    fitness_matrix, 
    num_selected=6, 
    seed=42
)

print(f"Selected individuals: {selected}")
print(f"Selection frequency: {np.bincount(selected, minlength=4)}")


Selected individuals: [3 2 0 0 2 3]
Selection frequency: [2 0 2 2]


## Epsilon Lexicase Selection

Epsilon lexicase allows individuals within a tolerance (epsilon) of the best performance to be considered equally good, promoting diversity.


In [6]:
# Select with epsilon tolerance
selected_eps = lexicase.epsilon_lexicase_selection(
    fitness_matrix, 
    num_selected=6, 
    epsilon=1.0,  # Individuals within 1.0 of best are considered equal
    seed=42
)

print(f"Epsilon lexicase selected: {selected_eps}")
print(f"Selection frequency: {np.bincount(selected_eps, minlength=4)}")


Epsilon lexicase selected: [2 2 0 0 0 2]
Selection frequency: [3 0 3 0]


## Summary

Lexicase selection evaluates individuals on test cases in random order, keeping only the best performers at each step. This creates selection pressure while maintaining diversity across different problem aspects.
